In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from beta_VAE import BetaVAE

selected_leads = ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVL", "LEAD_aVF"]

lead_file_paths = {
    "LEAD_I": "../../Data_processing/data_aspire_PAP/LEAD_I.pt",
    "LEAD_II": "../../Data_processing/data_aspire_PAP/LEAD_II.pt",
    "LEAD_III": "../../Data_processing/data_aspire_PAP/LEAD_III.pt",
    "LEAD_aVR": "../../Data_processing/data_aspire_PAP/LEAD_aVR.pt",
    "LEAD_aVL": "../../Data_processing/data_aspire_PAP/LEAD_aVL.pt",
    "LEAD_aVF": "../../Data_processing/data_aspire_PAP/LEAD_aVF.pt",
}

labels_file_path = "../../Data_processing/data_aspire_PAP/labels.pt"

# Load only the selected lead tensors and labels
ecg_lead_tensors = {lead: torch.load(path) for lead, path in lead_file_paths.items()}
labels = torch.load(labels_file_path)

# Ensure all leads have the same number of samples as the labels
sample_count = len(next(iter(ecg_lead_tensors.values())))
assert len(labels) == sample_count, "Mismatch between number of labels and samples."
for tensor in ecg_lead_tensors.values():
    assert len(tensor) == sample_count, "All leads must have the same number of samples."

# ✅ **Updated Dataset Class for 6-Lead ECG**
class ECGDataset2DWithLabels(Dataset):
    def __init__(self, ecg_leads, labels):
        self.ecg_leads = ecg_leads
        self.labels = labels

    def __len__(self):
        return len(next(iter(self.ecg_leads.values())))

    def __getitem__(self, idx):
        # **Ensure the shape is (6, 1, Sequence_Length)**
        stacked_leads = torch.stack([self.ecg_leads[lead][idx] for lead in self.ecg_leads]).unsqueeze(1)
        label = self.labels[idx]
        return stacked_leads, label  # Shape: (6, 1, sequence_length), label

# ✅ **Initialize dataset and dataloader with 6-Lead Data**
dataset = ECGDataset2DWithLabels(ecg_lead_tensors, labels)

In [2]:
import torch
import torch.nn as nn

class ECGClassifier(nn.Module):
    def __init__(self, pretrained_vae, num_classes=2):
        super(ECGClassifier, self).__init__()

        # **Freeze VAE encoder**
        for param in pretrained_vae.encoder.parameters():
            param.requires_grad = False


        self.encoder = pretrained_vae.encoder
        self.z_mean = pretrained_vae.z_mean  # Extract mu as features


        latent_dim = pretrained_vae.z_mean.out_features  # ✅ Get correct latent_dim

        # **Updated Classification Head**
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, 128),  # ✅ Use the correct latent_dim from z_mean output
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.encoder(x)  # Pass through frozen VAE encoder
        x = x.flatten(start_dim=1)  # **Flatten before passing to z_mean**
        x = self.z_mean(x)  # ✅ Extract latent features from z_mean
        logits = self.classifier(x)  # Pass through classifier
        return logits


In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef
import numpy as np
import random

# **Set device**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# **Set seed for reproducibility**
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(123)

pretrained_vae = BetaVAE(input_channels=6, sequence_length=5000, latent_dim=256).to(device)  # Updated to 6 leads
pretrained_vae.load_state_dict(torch.load("TS_VAE.pth", map_location=device))  # ✅ Load correct weights
pretrained_vae.eval()  # Set to evaluation mode

# **Training function**
def train_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            lead_data, labels = batch
            lead_data, labels = lead_data.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(lead_data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')


# **Evaluation function**
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            lead_data, labels = batch
            lead_data, labels = lead_data.to(device), labels.to(device)

            logits = model(lead_data)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    auc_score = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return accuracy, auc_score, f1, mcc


# **Cross-Validation Training**
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subset = Subset(dataset, train_ids)
    test_subset = Subset(dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=128, shuffle=False)

    # **Reinitialize classifier for each fold with 6-lead input**
    model = ECGClassifier(pretrained_vae, num_classes=2).to(device)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    # **Train classifier**
    train_classifier(model, train_loader, criterion, optimizer, epochs=50)

    # **Evaluate classifier**
    accuracy, auc_score, f1, mcc = evaluate_model(model, test_loader)
    fold_results.append((accuracy, auc_score, f1, mcc))

    print(f'Fold {fold} Results: Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}, F1: {f1:.4f}, MCC: {mcc:.4f}\n')

# **Calculate Metrics Across Folds**
accuracies, aucs, f1s, mccs = zip(*fold_results)
print(f'Mean Accuracy: {np.mean(accuracies):.4f}, STD: {np.std(accuracies):.4f}')
print(f'Mean AUC: {np.mean(aucs):.4f}, STD: {np.std(aucs):.4f}')
print(f'Mean F1: {np.mean(f1s):.4f}, STD: {np.std(f1s):.4f}')
print(f'Mean MCC: {np.mean(mccs):.4f}, STD: {np.std(mccs):.4f}')  # MCC Statistics


Task-2

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Define file paths for the 6-lead ECG data and labels
selected_leads = ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVL", "LEAD_aVF"]

lead_file_paths = {
    "LEAD_I": "../../Data_processing/data_aspire_PAWP/LEAD_I.pt",
    "LEAD_II": "../../Data_processing/data_aspire_PAWP/LEAD_II.pt",
    "LEAD_III": "../../Data_processing/data_aspire_PAWP/LEAD_III.pt",
    "LEAD_aVR": "../../Data_processing/data_aspire_PAWP/LEAD_aVR.pt",
    "LEAD_aVL": "../../Data_processing/data_aspire_PAWP/LEAD_aVL.pt",
    "LEAD_aVF": "../../Data_processing/data_aspire_PAWP/LEAD_aVF.pt",
}

labels_file_path = "../../Data_processing/data_aspire_PAWP/labels.pt"

# Load only the selected lead tensors and labels
ecg_lead_tensors = {lead: torch.load(path) for lead, path in lead_file_paths.items()}
labels = torch.load(labels_file_path)

# Ensure all leads have the same number of samples as the labels
sample_count = len(next(iter(ecg_lead_tensors.values())))
assert len(labels) == sample_count, "Mismatch between number of labels and samples."
for tensor in ecg_lead_tensors.values():
    assert len(tensor) == sample_count, "All leads must have the same number of samples."

# ✅ **Updated Dataset Class for 6-Lead ECG**
class ECGDataset2DWithLabels(Dataset):
    def __init__(self, ecg_leads, labels):
        self.ecg_leads = ecg_leads
        self.labels = labels

    def __len__(self):
        return len(next(iter(self.ecg_leads.values())))

    def __getitem__(self, idx):
        # **Ensure the shape is (6, 1, Sequence_Length)**
        stacked_leads = torch.stack([self.ecg_leads[lead][idx] for lead in self.ecg_leads]).unsqueeze(1)
        label = self.labels[idx]
        return stacked_leads, label  # Shape: (6, 1, sequence_length), label

# ✅ **Initialize dataset and dataloader with 6-Lead Data**
dataset = ECGDataset2DWithLabels(ecg_lead_tensors, labels)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# ✅ **Test Shape of a Single Batch**
batch = next(iter(dataloader))
lead_data, label = batch
print(f"Batch size: {len(label)}")
print(f"Shape of input: {lead_data.shape}")  # **Expected: (Batch, 6, 1, Sequence_Length)**
print(f"Labels: {label}")


In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef
import numpy as np
import random

# **Set device**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# **Set seed for reproducibility**
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

pretrained_vae = BetaVAE(input_channels=6, sequence_length=5000, latent_dim=256).to(device)  # Updated to 6 leads
pretrained_vae.load_state_dict(torch.load("TS_VAE.pth", map_location=device))  # ✅ Load correct weights
pretrained_vae.eval()  # Set to evaluation mode

# **Training function**
def train_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            lead_data, labels = batch
            lead_data, labels = lead_data.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(lead_data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')


# **Evaluation function**
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            lead_data, labels = batch
            lead_data, labels = lead_data.to(device), labels.to(device)

            logits = model(lead_data)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    auc_score = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return accuracy, auc_score, f1, mcc


# **Cross-Validation Training**
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subset = Subset(dataset, train_ids)
    test_subset = Subset(dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=128, shuffle=False)

    # **Reinitialize classifier for each fold with 6-lead input**
    model = ECGClassifier(pretrained_vae, num_classes=2).to(device)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    # **Train classifier**
    train_classifier(model, train_loader, criterion, optimizer, epochs=50)

    # **Evaluate classifier**
    accuracy, auc_score, f1, mcc = evaluate_model(model, test_loader)
    fold_results.append((accuracy, auc_score, f1, mcc))

    print(f'Fold {fold} Results: Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}, F1: {f1:.4f}, MCC: {mcc:.4f}\n')

# **Calculate Metrics Across Folds**
accuracies, aucs, f1s, mccs = zip(*fold_results)
print(f'Mean Accuracy: {np.mean(accuracies):.4f}, STD: {np.std(accuracies):.4f}')
print(f'Mean AUC: {np.mean(aucs):.4f}, STD: {np.std(aucs):.4f}')
print(f'Mean F1: {np.mean(f1s):.4f}, STD: {np.std(f1s):.4f}')
print(f'Mean MCC: {np.mean(mccs):.4f}, STD: {np.std(mccs):.4f}')  # MCC Statistics


Task-3

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
# File paths
lead_file_paths = {
    "LEAD_I": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_I.pt",
    "LEAD_II": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_II.pt",
    "LEAD_III": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_III.pt",
    "LEAD_aVR": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_aVR.pt",
    "LEAD_aVL": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_aVL.pt",
    "LEAD_aVF": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_aVF.pt",
    "LEAD_V1": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V1.pt",
    "LEAD_V2": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V2.pt",
    "LEAD_V3": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V3.pt",
    "LEAD_V4": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V4.pt",
    "LEAD_V5": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V5.pt",
    "LEAD_V6": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V6.pt"
}
labels_file_path = "D:/ukbiobank/ECG_PAWP_UKB_Final/labels.pt"

# Load all lead tensors and labels
ecg_lead_tensors = {lead: torch.load(path) for lead, path in lead_file_paths.items()}
labels = torch.load(labels_file_path)

# Ensure all leads have the same number of samples as the labels
sample_count = len(next(iter(ecg_lead_tensors.values())))
print(len(labels))
print(sample_count)
assert len(labels) == sample_count, "Mismatch between number of labels and samples."
for tensor in ecg_lead_tensors.values():
    assert len(tensor) == sample_count, "All leads must have the same number of samples."

# Define the dataset class
class ECGMultiLeadDatasetWithLabels(Dataset):
    def __init__(self, ecg_leads, labels):
        self.ecg_leads = ecg_leads
        self.labels = labels

    def __len__(self):
        return len(next(iter(self.ecg_leads.values())))

    def __getitem__(self, idx):
        # Return each lead sample with the correct input shape and corresponding label
        lead_data = {lead: self.ecg_leads[lead][idx].unsqueeze(0) for lead in self.ecg_leads}
        label = self.labels[idx]
        return lead_data, label

# Initialize the dataset and dataloader
dataset = ECGMultiLeadDatasetWithLabels(ecg_lead_tensors, labels)

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef
import numpy as np
import random

# **Set device**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# **Set seed for reproducibility**
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

pretrained_vae = BetaVAE(input_channels=6, sequence_length=5000, latent_dim=256).to(device)  # Updated to 6 leads
pretrained_vae.load_state_dict(torch.load("TS_VAE.pth", map_location=device))  # ✅ Load correct weights
pretrained_vae.eval()  # Set to evaluation mode

# **Training function**
def train_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            lead_data, labels = batch
            lead_data, labels = lead_data.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(lead_data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')


# **Evaluation function**
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            lead_data, labels = batch
            lead_data, labels = lead_data.to(device), labels.to(device)

            logits = model(lead_data)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    auc_score = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return accuracy, auc_score, f1, mcc


# **Cross-Validation Training**
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subset = Subset(dataset, train_ids)
    test_subset = Subset(dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=128, shuffle=False)

    # **Reinitialize classifier for each fold with 6-lead input**
    model = ECGClassifier(pretrained_vae, num_classes=2).to(device)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    # **Train classifier**
    train_classifier(model, train_loader, criterion, optimizer, epochs=50)

    # **Evaluate classifier**
    accuracy, auc_score, f1, mcc = evaluate_model(model, test_loader)
    fold_results.append((accuracy, auc_score, f1, mcc))

    print(f'Fold {fold} Results: Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}, F1: {f1:.4f}, MCC: {mcc:.4f}\n')

# **Calculate Metrics Across Folds**
accuracies, aucs, f1s, mccs = zip(*fold_results)
print(f'Mean Accuracy: {np.mean(accuracies):.4f}, STD: {np.std(accuracies):.4f}')
print(f'Mean AUC: {np.mean(aucs):.4f}, STD: {np.std(aucs):.4f}')
print(f'Mean F1: {np.mean(f1s):.4f}, STD: {np.std(f1s):.4f}')
print(f'Mean MCC: {np.mean(mccs):.4f}, STD: {np.std(mccs):.4f}')  # MCC Statistics
